# **기업과제4_10팀**.ipynb
### Contents
* Installation
* Data Preprocessing
* Train
* Validation
* Inference
* Rouge Score

본 노트북은 채택한 **요약 모델의 전반적인 학습과정**을 담고 있습니다. 모델을 Rouge-1 score을 이용해 정량적으로 평가한 모듈을 원하시면 `metric.py`을,   
모델이 실제로 본문의 주제를 잘 감지하고 있는지 정성적으로 검증한 결과를 원하시면 `기업과제4_10팀_LSATF_IDFSVD.ipynb`확인해주시길 바랍니다.
```
⚓ 원티드 프리온보딩 10팀 / 강상훈, 권예환, 남준우, 장수림, 조성빈
```

# Installation

In [ ]:
!git clone https://github.com/aiKSTlab/kobertsum.git

Cloning into 'kobertsum'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 189 (delta 78), reused 127 (delta 37), pack-reused 0
Receiving objects: 100% (189/189), 15.07 MiB | 8.20 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [ ]:
%cd kobertsum
!python main.py -task install

/content/kobertsum
/content/kobertsum
     |████████████████████████████████| 1.2 MB 15.2 MB/s 
     |████████████████████████████████| 60 kB 10.3 MB/s 
     |████████████████████████████████| 2.1 MB 86.2 MB/s 
     |████████████████████████████████| 125 kB 85.9 MB/s 
     |████████████████████████████████| 895 kB 69.5 MB/s 
     |████████████████████████████████| 3.3 MB 70.6 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191621 sha256=5d171750362cd786def78357be699380c7dfadeabbe946ac8f0853d5e6f299a1
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge
-----------------------mecab 설치--------------------------
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  

In [ ]:
!pip install jsonlines

In [ ]:
!pip install transformers

In [ ]:
!pip install rouge

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re

import math
import json
import random
import jsonlines

import pandas as pd
import numpy as np
from itertools import chain

from rouge import Rouge
from bs4 import BeautifulSoup

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn.init import xavier_uniform_

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split

from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
# seed
seed = 7777
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


# Data Preprocessing
#### Dataset : Aihub 문서요약 텍스트 데이터셋
#### URI : https://aihub.or.kr/aidata/8054

In [ ]:
with open('/content/drive/MyDrive/wanted/train_original.json', 'r') as f:
    ext_train = json.load(f)

with open('/content/drive/MyDrive/wanted/valid_original.json', 'r') as f:
    ext_valid = json.load(f)

In [ ]:
train_data = pd.DataFrame(ext_train)["documents"]
train_data = pd.DataFrame([x for x in train_data])

In [ ]:
train_data.head()

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...
3,290741794,정치,online,지역지,광양신문,small,746,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화..."
4,290741797,종합,online,지역지,광양신문,small,978,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...


In [ ]:
for idx, i in enumerate(train_data["extractive"]):
  if None in i:
    print(idx)
    print(i)

21810
[None, 2, 1]
50279
[None, 3, 4]
61894
[4, None, 7]
67953
[None, None, None]
68553
[None, None, None]
184704
[None, 2, 3]


In [ ]:
train_data.drop([21810, 50279, 61894, 67953, 68553, 184704], inplace=True)

In [ ]:
train_data["text"] = train_data["text"].apply(lambda x : [x["sentence"] for x in list(chain(*x))])
train_data["abstractive"] = train_data["abstractive"].apply(lambda x : x[0])

train_data = train_data[["extractive", "text", "abstractive"]]
train_data.rename(columns = {'text' : 'article_original'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train_data.head()

,extractive,article_original,abstractive
0,"[2, 3, 10]","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"[2, 4, 11]","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,"[3, 5, 7]","[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"[2, 3, 4]","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"[3, 7, 4]","[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [ ]:
valid_data = pd.DataFrame(ext_valid)["documents"]
valid_data = pd.DataFrame([x for x in valid_data])

valid_data["text"] = valid_data["text"].apply(lambda x : [x["sentence"] for x in list(chain(*x))])
valid_data["abstractive"] = valid_data["abstractive"].apply(lambda x : x[0])
valid_data = valid_data[["extractive", "text", "abstractive"]]
valid_data.rename(columns = {'text' : 'article_original'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
for idx, i in enumerate(valid_data["extractive"]):
  if None in i:
    print(idx)
    print(i)

4318
[None, 3, 4]


In [ ]:
valid_data.drop(4318, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
valid_data.head()

,extractive,article_original,abstractive
0,"[0, 1, 3]","[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,"[2, 1, 6]","[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,"[1, 5, 8]","[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,"[0, 1, 2]",[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[1, 3, 4]","[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [ ]:
train_json_dict = [{"article_original": x, 
                    "extractive": y,
                    "abstractive": z} for x,y,z in zip(train_data["article_original"], train_data["extractive"], train_data["abstractive"])]

valid_json_dict = [{"article_original": x, 
                    "extractive": y,
                    "abstractive": z} for x,y,z in zip(valid_data["article_original"], valid_data["extractive"], valid_data["abstractive"])]

In [ ]:
%mkdir my_data

with open("./my_data/train.jsonl", encoding= "utf-8",mode="w") as file:
  for i in train_json_dict:
    file.write(json.dumps(i) + "\n")

with open("./my_data/valid.jsonl", encoding= "utf-8",mode="w") as file:
  for i in valid_json_dict:
    file.write(json.dumps(i) + "\n")

In [ ]:
!python preprocess_data.py -target_summary_sent ext -model_path monologg/koelectra-base-v3-discriminator -n_cpus 16 -train_data_name train.jsonl -test_data_name valid.jsonl -data_path ./my_data -save_path ./data

train_df(231778) is exported
valid_df(12199) is exported
test_df(30121) is exported
  0% 0/232 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
  0% 1/232 [00:01<04:47,  1.24s/it]/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'/'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
  5% 12/232 [00:16<04:45,  1.30s/it]/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.yasl.co.kr/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
 16% 36/232 [00:47<04:13, 

# Train

In [ ]:
!python main_function.py -mode train -task ext \
-model_path monologg/koelectra-base-v3-discriminator \
-save_path /content/drive/MyDrive/wanted/giup4/model \
-data_path /content/drive/MyDrive/wanted/giup4/ext_data \
-save_checkpoint_steps 10000 \
-visible_gpus 0 -report_every 500 \
-ext_dropout 0.1 -max_pos 512 -lr 2e-3 -warmup_steps 100 \
-batch_size 5000 -accum_count 2 \
-train_steps 20000  -use_interval true

log_dir :  ./logs
[2022-03-22 18:04:06,873 INFO] Device ID 0
[2022-03-22 18:04:06,874 INFO] Device cuda
[2022-03-22 18:04:16,804 INFO] ExtSummarizer(
  (bert): Bert(
    (model): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(35000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0): ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
   

# Validation
### loss = xent(CrossEntropy)

In [ ]:
!python main_function.py -mode valid -task ext \
 -test_all True \
 -model_path /content/drive/MyDrive/wanted/giup4/model \
 -model_pt model_step_20000.pt \
 -data_path /content/drive/MyDrive/wanted/giup4/ext_data \
 -result_path ./result \
 -log_dir ./logs \
 -test_batch_size 100  -batch_size 3000 \
 -sep_optim true \
 -use_interval true \
 -visible_gpus 0 -max_pos 512 -max_length 200 \
 -alpha 0.95 -min_length 50 \
 -report_rouge False -max_tgt_len 100

log_dir :  ./logs
[2022-03-23 05:37:20,165 INFO] Loading checkpoint from /content/drive/MyDrive/wanted/giup4/model/model_step_20000.pt
Namespace(accum_count=1, alpha=0.95, batch_size=3000, beam_size=5, bert_data_path='/content/drive/MyDrive/wanted/giup4/ext_data', beta1=0.9, beta2=0.999, block_trigram=True, data_path='/content/drive/MyDrive/wanted/giup4/ext_data', dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_dir='./logs', log_file='./logs/valid_20220323.log', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=100, min_length=50, mode='valid', model_path='/content/drive/MyDrive/wanted/giup4/model', model_pt='model_step_200

# Inference

In [ ]:
sport_news = pd.read_csv("/content/drive/MyDrive/wanted/sports_news_data.csv")

In [ ]:
sport_news

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15
...,...,...,...
9072,"이제 홈팬 야유 받는 먹튀 선수, 차비 감독 조차 “그만 해라”",[스포탈코리아] 한재현 기자= FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는...,2022-02-07
9073,"[오피셜] 성남, 만 17세 유스 김지수와 준프로 계약",[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로...,2022-02-07
9074,"오베르마스, 아약스서 쫓겨난다...'여성 동료들에게 부적절한 메시지'",[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. ...,2022-02-07
9075,"바르사 차비 감독, 트라오레 데뷔전 활약에 깜짝...""몸이 야수 같아""",[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른...,2022-02-07


In [ ]:
sport_news[sport_news["CONTENT"].isna()]

,TITLE,CONTENT,PUBLISH_DT
4137,GOAL50 2021 투표하기,NaN,2021-11-02
5475,"[GOAL LIVE] '오직 익수' 안익수가 생각하는 팬의 의미 ""상당히 두려운 존재""",NaN,2021-12-04


In [ ]:
#sport_news.dropna(inplace = True)

In [ ]:
sport_news

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15
...,...,...,...
9072,"이제 홈팬 야유 받는 먹튀 선수, 차비 감독 조차 “그만 해라”",[스포탈코리아] 한재현 기자= FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는...,2022-02-07
9073,"[오피셜] 성남, 만 17세 유스 김지수와 준프로 계약",[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로...,2022-02-07
9074,"오베르마스, 아약스서 쫓겨난다...'여성 동료들에게 부적절한 메시지'",[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. ...,2022-02-07
9075,"바르사 차비 감독, 트라오레 데뷔전 활약에 깜짝...""몸이 야수 같아""",[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른...,2022-02-07


In [ ]:
sport_data = sport_news["CONTENT"].apply(lambda x : str(x).split(".")).apply(lambda x : [y for y in x if y not in '' and ' ' and ' ' and ' ' and " "])

In [ ]:
def HTMLCleaner(doc):
  """ 본문에서 불필요한 HTML Tags를 제거하고 문장을 나눕니다. """
  doc = re.sub('\n', '<p>', doc)                # Unify Newline Character
  doc = re.sub('\.', '.<p>', doc)
  soup = BeautifulSoup(doc, "html.parser")      # Handling HTML tags
  blocklist = ['strong']  # Remove specific tags
  for tag in blocklist:
    tagging_soup = soup.findAll(tag)
    for script in tagging_soup:
      script.decompose()
  clean_list = [sentence for sentence in soup.strings]
  return clean_list

In [ ]:
def DocCleaner(text):
  """ 개별 문서에 적용해 문서를 전처리하는 함수입니다. """
  doc = HTMLCleaner(text)
  if doc:
    if doc[0].startswith('['):                # Remove media source
      doc[0] = re.sub('^\[.*?\] ', '', doc[0])
      doc[0] = re.sub(r'^.*?= ', '', doc[0])  # Remove reporter name
  doc = ' '.join(doc)
  doc = re.sub('\\\'', '', doc)
  doc = re.sub('\<.*?\>', '', doc)
  return doc

In [ ]:
def Preprocessing(df, title, col='summary'):
  """ 전체 데이터프레임에 적용해 문서를 전처리하는 함수입니다. """
  # Handling the missing values
  # We'll use the title as the text and its summary. (제곧내!)
  df[col] = df[col].fillna(title)
  # Cleaning the documents
  df['SUMMARY'] = df[col].apply(lambda x : DocCleaner(x))

  return df

In [ ]:
sport_data

0       [[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가...
1       [[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어...
2       [[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다, \...
3       [[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄...
4       [[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다,  팬들은 만족...
                              ...                        
9072    [[스포탈코리아] 한재현 기자= FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레...
9073    [[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프...
9074    [[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다,...
9075    [[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치...
9076    [[스포탈코리아] 반진혁 기자= 이름값만 화려한 맨유 워스트 11이 공개됐다, \n...
Name: CONTENT, Length: 9077, dtype: object

In [ ]:
sport_json_dict = [{"id": idx,"article_original": x} for idx, x in enumerate(sport_data)]

In [ ]:
with open("/content/kobertsum/my_data/sport.jsonl", encoding= "utf-8",mode="w") as file:
  for i in sport_json_dict:
    json.dump(i, file, ensure_ascii=False)
    file.write("\n")

**inference.py 코드 변경**

before
```
result_df = pd.merge(test_df, pd.DataFrame(test_pred_list), how="left", left_index=True, right_index=True)
result_df['summary'] = result_df.apply(lambda row: ' '.join(list(np.array(row['article_original'])[row['sum_sents_idxes']])) , axis=1)
```

after
```
result_df = pd.merge(test_df, pd.DataFrame(test_pred_list), how="left", left_index=True, right_index=True)
res_sum = []

for idx, (i, j) in enumerate(zip(result_df["article_original"], result_df["sum_sents_idxes"] )):
  if len(i) == 1:
    res_sum.append(i[0])
  elif len(i) <= max(j):
    j[j.index(max(j))] = -1
    summary = '. '.join([i[x] for x in j])
    res_sum.append(summary)
  else:
    summary = '. '.join([i[x] for x in j])
    res_sum.append(summary)

df_sum = pd.DataFrame(res_sum, columns=["summary"])

result_df = pd.concat([result_df, df_sum], axis =1)
```

In [ ]:
!python Inference.py -input_file ./my_data/sport.jsonl -candidate_file ./result/valid_step_20000.candidate -output_path ./output -output_csv output.csv

In [ ]:
inference = pd.read_csv("/content/kobertsum/output/output.csv")

In [ ]:
inference

,id,summary
0,0,\n\n지난 8일(한국 시간) 잉글랜드와 덴마크는 유로 2020 4강에서 결승 티켓...
1,1,\n\n지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸...
2,2,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다. \n...
3,3,[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...
4,4,팬들은 만족스럽다는 반응이다. [스포탈코리아] FC 바르셀로나가 새 시즌 원정 유...
...,...,...
9072,9072,[스포탈코리아] 한재현 기자= FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는...
9073,9073,[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로...
9074,9074,[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. ...
9075,9075,[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른...


In [ ]:
final = pd.concat([sport_news, inference["summary"]], axis =1)

In [ ]:
final = Preprocessing(final, title=final['TITLE'])

In [ ]:
final.drop(['summary'],axis = 1, inplace = True)

In [ ]:
final.iloc[5835]["CONTENT"]

'[스포탈코리아] 김진선 기자= 요주아 키미히(바이에른 뮌헨)가 코로나 후유증을 겪고있다. 그라운드 복귀도 미뤄질 전망이다.\n \n뮌헨은 9일 공식 홈페이지를 통해 "최근 코로나19에 감염됐던 키미히가 올 시즌 남은 경기에 뛸 수 없을 것"이라고 알렸다. \n\n독일 \'빌트\'도 “키미히가 격리를 마치고 CT 촬영을 진행했는데 폐에 체액이 고여있는 것으로 확인됐다. 그는 심각한 후유증을 겪고 있다”고 전했다. \n\n이어 “현재 상태로는 프로 선수로서 뛰는 게 절대 불가능하다. 다음 주에 검사를 재진행해 체액이 사라지는 걸 확인한 후 정상 훈련이 가능할 것”이라고 설명했다.\n\n키미히가 겪고 있는 증상은 코로나 감염 후 생기는 후유증 중 하나로, 염증이 생기면서 폐에 물과 같은 체액이 고이는 증상이다. 체액이 고일 시 호흡 곤란이 찾아올 수 있어 각별한 주의가 필요하다. \n\n키미히는 축구계에서 백신 접종을 거부한 대표적인 선수다. 지난 10월 키미히는 "백신에 관한 장기적인 연구가 부족하다. 안전성 확신이 없어서 접종하지 않겠다"고 말하며 백신 접종에 부정적 의견을 던졌다. 그리고 지난달 말 코로나19 양성 판정을 받았다. \n\n키미히는 자신의 인스타그램을 통해 팬들에게 근황을 전했다. 그는 "긴 시간 자가격리를 끝내고 잘 지내고 있다. 하지만 훈련으로 복귀하는 건 당장 힘들 것으로 보인다. 현재 폐에 문제가 생겼다. 한동안 시간이 걸릴 것이다. 얼른 치료 잘 받고 팀에 빠르게 합류하겠다”라고 말했다.\n\n키미히는 이번 달 열리는 분데스리가 FSV 마인츠 05전(11일), VfB 슈투트가르트전(15일), VfL 볼프스부르크전(18)까지 최소 3경기에 결장한다. \n\n사진=게티이미지코리아'

In [ ]:
final.iloc[5835]["SUMMARY"]

'요주아 키미히(바이에른 뮌헨)가 코로나 후유증을 겪고있다.   그라운드 복귀도 미뤄질 전망이다.     뮌헨은 9일 공식 홈페이지를 통해 "최근 코로나19에 감염됐던 키미히가 올 시즌 남은 경기에 뛸 수 없을 것"이라고 알렸다'

In [ ]:
final

,TITLE,CONTENT,PUBLISH_DT,SUMMARY
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15,지난 8일(한국 시간) 잉글랜드와 덴마크는 유로 2020 4강에서 결승 티켓을 두고...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15,지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸다. ...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다. 영국 매체 ‘스포...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다. 이재성은...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15,팬들은 만족스럽다는 반응이다. [스포탈코리아] FC 바르셀로나가 새 시즌 원정 ...
...,...,...,...,...
9072,"이제 홈팬 야유 받는 먹튀 선수, 차비 감독 조차 “그만 해라”",[스포탈코리아] 한재현 기자= FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는...,2022-02-07,FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다. 바르사...
9073,"[오피셜] 성남, 만 17세 유스 김지수와 준프로 계약",[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로...,2022-02-07,한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다...
9074,"오베르마스, 아약스서 쫓겨난다...'여성 동료들에게 부적절한 메시지'",[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. ...,2022-02-07,레전드 마르크 오베르마스가 아약스에서 쫓겨났다. 이유는 굉장히 굴욕적이었다. ...
9075,"바르사 차비 감독, 트라오레 데뷔전 활약에 깜짝...""몸이 야수 같아""",[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른...,2022-02-07,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다. ...


In [ ]:
pororo_ext = pd.read_csv("/content/pororo_ext.csv")

In [ ]:
pororo_ext = Preprocessing(pororo_ext, title=final['TITLE'])

In [ ]:
pororo_ext

,Unnamed: 0,summary,SUMMARY
0,0,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...
1,1,지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸다. 이...,지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸다. ...
2,2,영국 매체 ‘스포츠 바이블’은 14일(한국 시간) 슈퍼컴퓨터가 예측한 2021/20...,영국 매체 ‘스포츠 바이블’은 14일(한국 시간) 슈퍼컴퓨터가 예측한 2021/20...
3,3,[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다. 3년 계약...
4,4,[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다....
...,...,...,...
9072,9072,바르사는 7일 오전 0시 15분(한국시간) 스페인 바르셀로나 캄노우에서 열린 아틀레...,바르사는 7일 오전 0시 15분(한국시간) 스페인 바르셀로나 캄노우에서 열린 아틀레...
9073,9073,[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로...,한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다...
9074,9074,[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. ...,레전드 마르크 오베르마스가 아약스에서 쫓겨났다. 아약스는 7일(한국시간) 공식 홈...
9075,9075,[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른...,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다. ...


In [ ]:
final = final.rename(columns={"SUMMARY": "PREDICT SUMMARY"})

In [ ]:
final = pd.concat([final, pororo_ext["SUMMARY"]], axis =1)

In [ ]:
final = final.rename(columns={"SUMMARY": "TRUE SUMMARY"})

# Rouge Score

In [ ]:
rouge = Rouge()

In [ ]:
final

,TITLE,CONTENT,PUBLISH_DT,PREDICT SUMMARY,TRUE SUMMARY
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15,지난 8일(한국 시간) 잉글랜드와 덴마크는 유로 2020 4강에서 결승 티켓을 두고...,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15,지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸다. ...,지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸다. ...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다. 영국 매체 ‘스포...,영국 매체 ‘스포츠 바이블’은 14일(한국 시간) 슈퍼컴퓨터가 예측한 2021/20...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다. 이재성은...,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다. 3년 계약...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15,팬들은 만족스럽다는 반응이다. [스포탈코리아] FC 바르셀로나가 새 시즌 원정 ...,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다....
...,...,...,...,...,...
9072,"이제 홈팬 야유 받는 먹튀 선수, 차비 감독 조차 “그만 해라”",[스포탈코리아] 한재현 기자= FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는...,2022-02-07,FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다. 바르사...,바르사는 7일 오전 0시 15분(한국시간) 스페인 바르셀로나 캄노우에서 열린 아틀레...
9073,"[오피셜] 성남, 만 17세 유스 김지수와 준프로 계약",[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로...,2022-02-07,한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다...,한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다...
9074,"오베르마스, 아약스서 쫓겨난다...'여성 동료들에게 부적절한 메시지'",[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. ...,2022-02-07,레전드 마르크 오베르마스가 아약스에서 쫓겨났다. 이유는 굉장히 굴욕적이었다. ...,레전드 마르크 오베르마스가 아약스에서 쫓겨났다. 아약스는 7일(한국시간) 공식 홈...
9075,"바르사 차비 감독, 트라오레 데뷔전 활약에 깜짝...""몸이 야수 같아""",[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른...,2022-02-07,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다. ...,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다. ...


In [ ]:
#metric.py 파일 모듈
def rouge_metric(df):
  y_true = list(df['TRUE SUMMARY'])
  y_pred = list(df['PREDICT SUMMARY'])

  rouge = Rouge()
  score = [rouge.get_scores(y_true[i], y_pred[i], avg=True)['rouge-1']['f'] for i in range(len(y_true))]
  df['SCORE'] = score
  
  return df

In [ ]:
final = rouge_metric(final)

In [ ]:
final

,TITLE,CONTENT,PUBLISH_DT,PREDICT SUMMARY,TRUE SUMMARY,SCORE
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15,지난 8일(한국 시간) 잉글랜드와 덴마크는 유로 2020 4강에서 결승 티켓을 두고...,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,0.411765
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15,지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸다. ...,지난달 시작된 유로와 코파 아메리카가 11일(한국 시간)을 끝으로 막을 내렸다. ...,0.472727
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다. 영국 매체 ‘스포...,영국 매체 ‘스포츠 바이블’은 14일(한국 시간) 슈퍼컴퓨터가 예측한 2021/20...,0.723404
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다. 이재성은...,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다. 3년 계약...,0.440000
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15,팬들은 만족스럽다는 반응이다. [스포탈코리아] FC 바르셀로나가 새 시즌 원정 ...,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다....,0.976744
...,...,...,...,...,...,...
9072,"이제 홈팬 야유 받는 먹튀 선수, 차비 감독 조차 “그만 해라”",[스포탈코리아] 한재현 기자= FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는...,2022-02-07,FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다. 바르사...,바르사는 7일 오전 0시 15분(한국시간) 스페인 바르셀로나 캄노우에서 열린 아틀레...,0.606061
9073,"[오피셜] 성남, 만 17세 유스 김지수와 준프로 계약",[스포탈코리아] 한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로...,2022-02-07,한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다...,한재현 기자=성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다...,0.617647
9074,"오베르마스, 아약스서 쫓겨난다...'여성 동료들에게 부적절한 메시지'",[스포탈코리아] 곽힘찬 기자= '레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. ...,2022-02-07,레전드 마르크 오베르마스가 아약스에서 쫓겨났다. 이유는 굉장히 굴욕적이었다. ...,레전드 마르크 오베르마스가 아약스에서 쫓겨났다. 아약스는 7일(한국시간) 공식 홈...,0.695652
9075,"바르사 차비 감독, 트라오레 데뷔전 활약에 깜짝...""몸이 야수 같아""",[스포탈코리아] 고윤준 기자= FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른...,2022-02-07,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다. ...,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다. ...,0.657534


In [ ]:
rouge.get_scores(final["TRUE SUMMARY"], final["PREDICT SUMMARY"], avg=True)

{'rouge-1': {'f': 0.6267859088631494,
  'p': 0.5995428783912263,
  'r': 0.6817702381885684},
 'rouge-2': {'f': 0.5907098084277587,
  'p': 0.563543577453982,
  'r': 0.6465414410744965},
 'rouge-l': {'f': 0.626439082659903,
  'p': 0.5992214495817018,
  'r': 0.681368267710407}}

#Save Data

In [ ]:
final.to_csv("기업과제4_10팀_sports_news_data.csv",encoding="utf8")